In [1]:
import sys
sys.path.append('/home/prasse/work/Projekte/AEye/reading-comprehension/')
import os
import tensorflow as tf
GPU = 1
# select graphic card
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
config = tf.compat.v1.ConfigProto(log_device_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
tf_session = tf.compat.v1.Session(config=config)
from rf_baseline  import rf_feature_extraction as rf_feature_extraction
import datetime
from typing import Tuple
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import json
import random
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow import Variable
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Lambda
from tensorflow.math import l2_normalize
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPool1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import clone_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Embedding, Average, Dropout

%pylab inline
%load_ext autoreload
%autoreload 2

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0

Populating the interactive namespace from numpy and matplotlib


/usr/local/apps/Anaconda3/2020.11/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
pos_ids = np.where(y_train_all[:,label_dict['acc_level']] == 1)[0]
neg_ids = np.where(y_train_all[:,label_dict['acc_level']] == 0)[0]
for pos_id in pos_ids:
    x_instance = x_train_all[pos_id,:,-2]
    y_instance = x_train_all[pos_id,:,-1]
    use_ids = np.logical_and(x_instance != 0,
                            y_instance != 0)
    x_mean = np.mean(x_instance[use_ids])
    y_mean = np.mean(y_instance[use_ids])
    plt.plot(x_mean,y_mean,'rx')

for neg_id in neg_ids:
    x_instance = x_train_all[neg_id,:,-2]
    y_instance = x_train_all[neg_id,:,-1]
    use_ids = np.logical_and(x_instance != 0,
                            y_instance != 0)
    x_mean = np.mean(x_instance[use_ids])
    y_mean = np.mean(y_instance[use_ids])
    plt.plot(x_mean,y_mean,'bx')
plt.show()

NameError: name 'y_train_all' is not defined

In [3]:
def get_nn_model(ent_len,num_ent,
                 pos_len,num_pos,
                 num_features,input_len_fix=398,num_fix=4):
    
    drop_outs = 0.5
    
    x_ent_input = Input(shape=ent_len)
    x_ent = Embedding(input_dim = num_ent,
                        output_dim = 64)(x_ent_input)
    x_ent_avg = GlobalAveragePooling1D()(x_ent)
    x_ent_max = GlobalMaxPool1D()(x_ent)
    x_ent = Concatenate()([x_ent_avg,x_ent_max])
    x_ent = Dropout(0.2)(x_ent)
    x_ent = Dense(64,activation='relu')(x_ent)
    x_ent = Dense(32,activation='relu')(x_ent)


    x_pos_input = Input(shape=pos_len)
    x_pos = Embedding(input_dim = num_pos,
                        output_dim = 128)(x_pos_input)
    x_pos_avg = GlobalAveragePooling1D()(x_pos)
    x_pos_max = GlobalMaxPool1D()(x_pos)
    x_pos = Concatenate()([x_pos_avg,x_pos_max])
    x_pos = Dropout(0.2)(x_pos)
    x_pos = Dense(128,activation='relu')(x_pos)
    x_pos = Dense(64,activation='relu')(x_pos)
    x_pos = Dense(32,activation='relu')(x_pos)


    x_num_input = Input(shape=num_features)
    x_num = Dense(256,activation='relu')(x_num_input)
    x_num = Dropout(0.2)(x_num)
    x_num = Dense(128,activation='relu')(x_num)
    x_num = Dropout(drop_outs)(x_num)
    x_num = Dense(64,activation='relu')(x_num)
    x_num = Dropout(drop_outs)(x_num)
    x_num = Dense(32,activation='relu')(x_num)
    
    x_fix_input = Input(shape=(input_len_fix, num_fix))
    x_fix = Conv1D(64, 5, activation='gelu')(x_fix_input)
    x_fix = AveragePooling1D(pool_size=2)(x_fix)
    x_fix = BatchNormalization()(x_fix)
    x_fix_avg = GlobalAveragePooling1D()(x_fix)
    x_fix_max = GlobalMaxPool1D()(x_fix)
    x_fix = Concatenate()([x_fix_avg,x_fix_max])
    x_fix = Dropout(drop_outs)(x_fix)
    x_fix = Dense(32,activation='relu')(x_fix)

    x_concat = Concatenate()([x_ent,x_pos,x_num,x_fix])
    x_output = Dense(32,activation='relu')(x_concat)
    x_output = Dense(1,activation='sigmoid')(x_output)
    model = Model(inputs=[
                x_ent_input,
                x_pos_input,
                x_num_input,
                x_fix_input],
                outputs=x_output)
    opt = Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', auroc])
    #model.summary()
    return model

In [4]:
def help_roc_auc(y_true,y_pred):
    if len(np.unique(y_true)) == 1:
        return .5
    else:
        return roc_auc_score(y_true,y_pred)

# calculate the roc-auc as a metric
def auroc(y_true, y_pred):
    return tf.py_function(help_roc_auc, (y_true, y_pred), tf.double)

In [5]:
split_criterion = 'subj'
#split_criterion = 'book-page'
#split_criterion = 'book'
label = 'acc_level'
#label = 'native'
#label = 'subj_acc_level'
#label = 'difficulty'
normalize_flag = True
grid_search = False
use_gaze_entropy_features = True
input_window = 'page'
num_words = 150

patience = 50
batch_size = 256
epochs = 1000

In [6]:
spit_criterions = ['subj','book-page','book']
labels = ['acc_level','native','difficulty','subj_acc_level']

model_name = 'nn_paul'
axis = 1
n_estimators = 100

feature_names_per_word = [      'ff',
                                'tf',
                                'wfc_ff_nomarlized',
                                'wfc_ff_normalized',
                                'sc_ff_normalized',
                                'sc_tf_normalized',
                                'ic_ff_normalized',
                                'regression',
                                'num_regressions',
                                'num_progressions',
                                'surprisal',
                                'word_len']
    

for entity in rf_feature_extraction.entity_list:
    feature_names_per_word.append('is_entity_' + entity)

for pos in rf_feature_extraction.pos_list:
    feature_names_per_word.append('is_pos_' + pos)

feature_names_per_word += ['x_mean','y_mean']

feature_string = '_text_sequence_'
feature_string = '_text_sequence_with_paul_david_cluster_'
feature_string = '_text_sequence_with_paul_david_cluster_41_feature_'

for split_criterion in spit_criterions:
    for label in labels:
        save_path=f'/home/reich3/tmp/results/reading-comprehension/{model_name}_{split_criterion}_text_sequence_{label}_{input_window}_1d_conv.csv'

        SB_SAT_PATH = f'/raid/projects/reich3/reading-comprehension/paper_splits/{split_criterion}/'
        split_criterion_dict = {
            'subj': 0,
            'book': 1,
        }
        with open('/raid/projects/reich3/reading-comprehension/paper_splits/labels_dict.json') as fp:
            label_dict = json.load(fp)

        if split_criterion == 'book':
            num_folds = 4
        else:
            num_folds = 5
        pd_init = pd.DataFrame(
            columns=[
                'ahn_baseline',
                'fold0_auc', 'fold1_auc', 'fold2_auc', 'fold3_auc', 'fold4_auc',
                'fold0_tpr', 'fold1_tpr', 'fold2_tpr', 'fold3_tpr', 'fold4_tpr',
                'fold0_fpr', 'fold1_fpr', 'fold2_fpr', 'fold3_fpr', 'fold4_fpr',
                'fold0_y_pred', 'fold1_y_pred', 'fold2_y_pred', 'fold3_y_pred', 'fold4_y_pred',
                'fold0_y_test', 'fold1_y_test', 'fold2_y_test', 'fold3_y_test', 'fold4_y_test',
            'avg_auc', 'std_auc']
        )
        pd_init['ahn_baseline'] = [model_name]

        if input_window == 'window':
            input_shape = (21, )
        elif input_window == 'page':
            input_shape = (150, 11)




        feature_importances = []
        for fold in range(num_folds):
            np.random.seed(fold)
            random.seed(fold)
            X_train_path = os.path.join(SB_SAT_PATH, f'X_train_{split_criterion}{feature_string}{fold}.npy')
            X_train_fix_path = os.path.join(SB_SAT_PATH, f'X_train_{split_criterion}{feature_string}{fold}_fix_data.npy')
            y_train_path = os.path.join(SB_SAT_PATH, f'y_train_{split_criterion}{feature_string}{fold}.npy')
            x_train_all, y_train_all = np.load(X_train_path), np.load(y_train_path, allow_pickle=True)
            x_train_fix_all = np.load(X_train_fix_path)
            if normalize_flag:
                scaler = MinMaxScaler()
                fix_scaler = MinMaxScaler()
                x_train_all = scaler.fit_transform(x_train_all.reshape(-1, x_train_all.shape[-1])).reshape(x_train_all.shape)
                x_train_fix_all = fix_scaler.fit_transform(x_train_fix_all.reshape(-1, x_train_fix_all.shape[-1])).reshape(x_train_fix_all.shape)
            
            if split_criterion != 'book':
                outer_cv = KFold(n_splits=4, shuffle=True, random_state=fold)
            else:
                outer_cv = KFold(n_splits=3, shuffle=True, random_state=fold)

            if split_criterion != 'book-page':
                splitkeys = np.array(sorted(list(set(y_train_all[:, split_criterion_dict[split_criterion]]))))
            else:
                splitkeys = y_train_all[:, label_dict[label]]

            for train_idx, val_idx in outer_cv.split(splitkeys):
                break

            if split_criterion != 'book-page':
                N_train_sub = splitkeys[train_idx]
                N_test_sub = splitkeys[val_idx]
                
                train_idx = np.where(np.isin(y_train_all[:, split_criterion_dict[split_criterion]], N_train_sub))[0]
                val_idx = np.where(np.isin(y_train_all[:, split_criterion_dict[split_criterion]], N_test_sub))[0]
            x_train = x_train_all[train_idx]
            x_train_fix = x_train_fix_all[train_idx]
            y_train = y_train_all[train_idx]
            x_val = x_train_all[val_idx]
            x_val_fix = x_train_fix_all[val_idx]
            y_val = y_train_all[val_idx] 
            
            y_train = np.array(y_train[:, label_dict[label]], dtype=int)
            y_val = np.array(y_val[:, label_dict[label]], dtype=int)

            x_train, feature_names = rf_feature_extraction.get_rf_features_for_word_features(
                        x_train,feature_names_per_word,disable = False,
                        use_gaze_entropy_features = use_gaze_entropy_features)
            x_val, _ = rf_feature_extraction.get_rf_features_for_word_features(
                        x_val,feature_names_per_word,disable = False,
                        use_gaze_entropy_features = use_gaze_entropy_features)
            
            
            # Test Data
            X_test_path = os.path.join(SB_SAT_PATH, f'X_test_{split_criterion}{feature_string}{fold}.npy')
            y_test_path = os.path.join(SB_SAT_PATH, f'y_test_{split_criterion}{feature_string}{fold}.npy')
            X_test_fix_path = os.path.join(SB_SAT_PATH, f'X_test_{split_criterion}{feature_string}{fold}_fix_data.npy')
            x_test_all, y_test_all = np.load(X_test_path), np.load(y_test_path, allow_pickle=True)
            x_test_fix_all = np.load(X_test_fix_path)
            if normalize_flag:
                x_test_all = scaler.transform(x_test_all.reshape(-1, x_test_all.shape[-1])).reshape(x_test_all.shape)
                x_test_fix_all = fix_scaler.transform(x_test_fix_all.reshape(-1, x_test_fix_all.shape[-1])).reshape(x_test_fix_all.shape)
            
            y_test = np.array(y_test_all[:, label_dict[label]], dtype=int)


            x_test, _ = rf_feature_extraction.get_rf_features_for_word_features(
                        x_test_all,feature_names_per_word,disable = False,
                        use_gaze_entropy_features = use_gaze_entropy_features)
            
            
            # POS + Entity Lists
            pos_list_list_train, entity_list_list_train, pos_feature_names, entity_feature_names = rf_feature_extraction.get_watched_pos_entity_lists(
                    x_train_all,feature_names_per_word)
            pos_list_list_test, entity_list_list_test, _, _ = rf_feature_extraction.get_watched_pos_entity_lists(
                    x_test_all,feature_names_per_word)
            
            pos_lens     = [len(pos_list) for pos_list in pos_list_list_train]
            entity_lens  = [len(ent_list) for ent_list in entity_list_list_train]
            max_pos_len  = np.max(pos_lens)
            max_ent_len  = np.max(entity_lens)
            
            train_pos_matrix, pos_mapping_dict = rf_feature_extraction.convert_to_sequence_to_nn_input(
                            pos_list_list_train,pos_feature_names,max_pos_len)
            test_pos_matrix, _ = rf_feature_extraction.convert_to_sequence_to_nn_input(
                            pos_list_list_test,pos_feature_names,max_pos_len,pos_mapping_dict)
            train_ent_matrix, ent_mapping_dict = rf_feature_extraction.convert_to_sequence_to_nn_input(
                            entity_list_list_train,entity_feature_names,max_ent_len)
            test_ent_matrix, _ = rf_feature_extraction.convert_to_sequence_to_nn_input(
                            entity_list_list_test,entity_feature_names,max_ent_len,ent_mapping_dict)
            
            
            num_pos      = len(pos_mapping_dict) + 1
            num_ent      = len(ent_mapping_dict) + 1
            
            



            
            val_pos_matrix   = train_pos_matrix[val_idx]
            train_pos_matrix = train_pos_matrix[train_idx]
            test_pos_matrix  = test_pos_matrix
            
            val_ent_matrix   = train_ent_matrix[val_idx]
            train_ent_matrix = train_ent_matrix[train_idx]
            test_ent_matrix  = test_ent_matrix
            
            
            ent_len = train_ent_matrix.shape[1]
            pos_len = train_pos_matrix.shape[1]
            num_features = x_train.shape[1]
            model = get_nn_model(ent_len,num_ent,
                                 pos_len,num_pos,
                                 num_features)
            
            
            # scale the RF input
            rf_input_scaler = MinMaxScaler()
            x_train = rf_input_scaler.fit_transform(x_train)
            x_val = rf_input_scaler.transform(x_val)
            x_test = rf_input_scaler.transform(x_test)
            
            tf.keras.backend.clear_session()
            callbacks = [EarlyStopping(monitor='val_loss', patience=patience)]
            history = model.fit([train_ent_matrix,
                                 train_pos_matrix,
                                 x_train,x_train_fix],y_train,
                                         validation_data = (
                                             [val_ent_matrix,
                                              val_pos_matrix,
                                              x_val,x_val_fix],
                                             y_val),
                                         batch_size = batch_size,
                                         epochs = epochs,
                                         callbacks = callbacks,
                                         verbose = 0)
            

            y_pred = model.predict([test_ent_matrix,
                                      test_pos_matrix,
                                      x_test,x_test_fix_all],
                                   batch_size = batch_size)
            try:
                fpr, tpr, _ = metrics.roc_curve(
                    y_test,
                    y_pred,
                    pos_label=1,
                )
                auc = metrics.auc(fpr, tpr)
                print(split_criterion, label, auc)
                pd_init[f'fold{fold}_auc'] = auc
                pd_init[f'fold{fold}_tpr'] = [tpr]
                pd_init[f'fold{fold}_fpr'] = [fpr]
                pd_init[f'fold{fold}_y_test'] = [y_test]
                pd_init[f'fold{fold}_y_pred'] = [y_pred]
            except:
                try:
                    fpr, tpr, _ = metrics.roc_curve(
                        y_test,
                        y_pred,
                        pos_label=1,
                    )
                    auc = metrics.auc(fpr, tpr)
                    print(split_criterion, label, auc)
                    pd_init[f'fold{fold}_auc'] = auc
                    pd_init[f'fold{fold}_tpr'] = [tpr]
                    pd_init[f'fold{fold}_fpr'] = [fpr]
                    pd_init[f'fold{fold}_y_test'] = y_test
                    pd_init[f'fold{fold}_y_pred'] = y_pred
                except:
                    print(allo)

        pd_init['avg_auc'] = 0
        for i in range(num_folds):
            pd_init['avg_auc'] += pd_init[f'fold{i}_auc']
        pd_init['avg_auc']  /= num_folds

        pd_init['std_auc'] = 0
        for i in range(0, num_folds):
            pd_init['std_auc'] +=  (pd_init[f'fold{i}_auc'] - pd_init['avg_auc'])**2
        pd_init['std_auc'] = (pd_init['std_auc']/num_folds)**(1/2)
        pd_init.to_csv(save_path, index=None)
        print(f"mean auc: {pd_init['avg_auc'].values[0]:.6f}\pm{pd_init['std_auc'].values[0]:.6f}")
        #print(allo)

FileNotFoundError: [Errno 2] No such file or directory: '/raid/projects/reich3/reading-comprehension/paper_splits/subj/X_train_subj_text_sequence_with_paul_david_cluster_41_feature_0.npy'

In [ ]:
pd_init.head()

In [ ]:
train_ent_matrix.shape

In [ ]:
train_pos_matrix.shape

In [ ]:
x_train.shape

In [ ]:
model = get_nn_model(ent_len,pos_len,num_features,num_pos,num_features)
            
callbacks = [EarlyStopping(monitor='val_loss', patience=patience)]

history_val = model.fit([train_ent_matrix,
                     train_pos_matrix,
                     x_train],y_train,
                             validation_data = (
                                 [val_ent_matrix,
                                  val_pos_matrix,
                                  x_val],
                                 y_val),
                             batch_size = batch_size,
                             epochs = epochs,
                             callbacks = callbacks,
                             verbose = 1)

model = get_nn_model(ent_len,pos_len,num_features)

history_test = model.fit([train_ent_matrix,
                     train_pos_matrix,
                     x_train],y_train,
                             validation_data = (
                                 [test_ent_matrix,
                                  test_pos_matrix,
                                  x_test],
                                 y_test),
                             batch_size = batch_size,
                             epochs = epochs,
                             callbacks = callbacks,
                             verbose = 1)

In [ ]:
plt.plot(np.arange(len(history_val.history['val_auroc'])),history_val.history['val_auroc'],label='validation')
plt.plot(np.arange(len(history_val.history['val_auroc'])),history_val.history['auroc'],label='train')
plt.xlabel('epochs')
plt.ylabel('ROC AUC')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(len(history_test.history['val_auroc'])),history_test.history['val_auroc'],label='validation')
plt.plot(np.arange(len(history_test.history['val_auroc'])),history_test.history['auroc'],label='train')
plt.xlabel('epochs')
plt.ylabel('ROC AUC')
plt.legend()
plt.show()

In [ ]:
history_test.history['val_auroc'][-1]

In [ ]:
y_pred = model.predict([test_ent_matrix,
                          test_pos_matrix,
                          x_test],
                       batch_size = batch_size)

fpr, tpr, _ = metrics.roc_curve(
    y_test,
    y_pred,
    pos_label=1,
)
auc = metrics.auc(fpr, tpr)
print(auc)

In [ ]:
import seaborn as sns
sns.distplot(y_pred)
plt.show()

In [ ]:
ent_len = train_ent_matrix.shape[1]
pos_len = train_pos_matrix.shape[1]
num_features = x_train.shape[1]
model = get_nn_model(ent_len,pos_len,num_features,num_pos,num_features)

In [ ]:
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['val_auroc'],label='validation')
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['auroc'],label='train')
plt.xlabel('epochs')
plt.ylabel('ROC AUC')
plt.legend()
plt.show()

### X Y input

In [ ]:
#x_train_inputs = x_train_all[:,:,-2]
#y_train_inputs = x_train_all[:,:,-1]
train_inputs   = x_train_all[:,:,[-2,-1]]
val_inputs     = train_inputs[val_idx]
train_inputs   = train_inputs[train_idx]
#x_test_inputs  = x_test_all[:,:,-2]
#y_test_inputs  = x_test_all[:,:,-2]
test_inputs    = x_test_all[:,:,[-2,-1]]

x_input = Input(shape=(train_inputs.shape[1],train_inputs.shape[2]))
x_avg = GlobalAveragePooling1D()(x_input)
x_max = GlobalMaxPool1D()(x_input)
x_output = Concatenate()([x_avg,x_max])
#x_output = Dense(256,activation='relu')(x_output)
#x_output = Dense(128,activation='relu')(x_output)
#x_output = Dropout(0.2)(x_output)
#x_output = Dense(64,activation='relu')(x_output)
#x_output = Dense(32,activation='relu')(x_output)
#x_output = Dense(16,activation='relu')(x_output)
x_output = Dense(1,activation='sigmoid')(x_output)
model = Model(inputs=x_input, outputs=x_output)
opt = Adam()
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', auroc])
model.summary()

patience = 50
batch_size = 256
epochs = 1000

callbacks = [EarlyStopping(monitor='val_auroc', patience=patience)]
history = model.fit(train_inputs,y_train,
                             validation_data = (val_inputs,y_val),
                             batch_size = batch_size,
                             epochs = epochs,
                             callbacks = callbacks,
                             verbose = 0)

plt.plot(np.arange(len(history.history['val_auroc'])),history.history['val_auroc'],label='validation')
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['auroc'],label='train')
plt.xlabel('epochs')
plt.ylabel('ROC AUC')
plt.legend()
plt.show()

print(history.history['val_auroc'][-1])

y_pred = model.predict(test_inputs,
                       batch_size = batch_size)

fpr, tpr, _ = metrics.roc_curve(
    y_test,
    y_pred,
    pos_label=1,
)
auc = metrics.auc(fpr, tpr)
print(auc)

In [ ]:
#x_train_inputs = x_train_all[:,:,-2]
#y_train_inputs = x_train_all[:,:,-1]
train_inputs   = x_train_all[:,:,[-2,-1]]
val_inputs     = train_inputs[val_idx]
train_inputs   = train_inputs[train_idx]
#x_test_inputs  = x_test_all[:,:,-2]
#y_test_inputs  = x_test_all[:,:,-2]
test_inputs    = x_test_all[:,:,[-2,-1]]

x_input = Input(shape=(train_inputs.shape[1],train_inputs.shape[2]))
x = Conv1D(32, 4, activation='gelu')(x_input)
x = AveragePooling1D(pool_size=2)(x)
x = BatchNormalization()(x)
x = Conv1D(64, 2, activation='gelu')(x)
x = AveragePooling1D(pool_size=2)(x)
x = BatchNormalization()(x)
#x = Conv1D(64, 4, activation='gelu')(x)
#x = AveragePooling1D(pool_size=2)(x)
#x = BatchNormalization()(x)
#x = Flatten()(x)
x = GlobalAveragePooling1D()(x)
#x = Dense(1024,activation='relu')(x)
#x = Dropout(0.5)(x)
#x = Dense(512,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32,activation='relu')(x)
x_output = Dense(1,activation='sigmoid')(x)
model = Model(inputs=x_input, outputs=x_output)
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', auroc])
model.summary()

patience = 5000
batch_size = 256
epochs = 1000

callbacks = [EarlyStopping(monitor='val_auroc', patience=patience)]
history = model.fit(train_inputs,y_train,
                             validation_data = (val_inputs,y_val),
                             batch_size = batch_size,
                             epochs = epochs,
                             callbacks = callbacks,
                             verbose = 0)

plt.plot(np.arange(len(history.history['val_auroc'])),history.history['val_auroc'],label='validation')
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['auroc'],label='train')
plt.xlabel('epochs')
plt.ylabel('ROC AUC')
plt.legend()
plt.show()

print(history.history['val_auroc'][-1])

y_pred = model.predict(test_inputs,
                       batch_size = batch_size)

fpr, tpr, _ = metrics.roc_curve(
    y_test,
    y_pred,
    pos_label=1,
)
auc = metrics.auc(fpr, tpr)
print(auc)

### POS input

In [ ]:
x_input = Input(shape=train_pos_matrix.shape[1])
x_output = Embedding(input_dim = num_pos,
                    output_dim = 128)(x_input)
x_avg = GlobalAveragePooling1D()(x_output)
x_max = GlobalMaxPool1D()(x_output)
x_output = Concatenate()([x_avg,x_max])
#x_output = Dense(256,activation='relu')(x_output)
x_output = Dense(128,activation='relu')(x_output)
#x_output = Dropout(0.2)(x_output)
x_output = Dense(64,activation='relu')(x_output)
x_output = Dense(32,activation='relu')(x_output)
x_output = Dense(16,activation='relu')(x_output)
x_output = Dense(1,activation='sigmoid')(x_output)
model = Model(inputs=x_input, outputs=x_output)
opt = Adam()
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', auroc])
model.summary()

patience = 50
batch_size = 256
epochs = 1000

callbacks = [EarlyStopping(monitor='val_auroc', patience=patience)]
history = model.fit(train_pos_matrix,y_train,
                             validation_data = (val_pos_matrix,y_val),
                             batch_size = batch_size,
                             epochs = epochs,
                             callbacks = callbacks,
                             verbose = 0)

plt.plot(np.arange(len(history.history['val_auroc'])),history.history['val_auroc'],label='validation')
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['auroc'],label='train')
plt.xlabel('epochs')
plt.ylabel('ROC AUC')
plt.legend()
plt.show()

print(history.history['val_auroc'][-1])

y_pred = model.predict(test_pos_matrix,
                       batch_size = batch_size)

fpr, tpr, _ = metrics.roc_curve(
    y_test,
    y_pred,
    pos_label=1,
)
auc = metrics.auc(fpr, tpr)
print(auc)

## POS 1DCNN

In [ ]:
x_input = Input(shape=train_pos_matrix.shape[1])
x_emb = Embedding(input_dim = num_pos,
                    output_dim = 32)(x_input)
x = Conv1D(64, 3, activation='gelu')(x_emb)
x = GlobalMaxPool1D()(x)
x = Flatten()(x)
x_avg = GlobalAveragePooling1D()(x_emb)
concat = Concatenate()([x,x_avg])
x_output = Dense(32,activation='relu')(concat)
x_output = Dense(1,activation='sigmoid')(x_output)
model = Model(inputs=x_input, outputs=x_output)
opt = Adam()
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', auroc])
model.summary()

patience = 50
batch_size = 256
epochs = 1000

callbacks = [EarlyStopping(monitor='val_auroc', patience=patience)]
history = model.fit(train_pos_matrix,y_train,
                             validation_data = (val_pos_matrix,y_val),
                             batch_size = batch_size,
                             epochs = epochs,
                             callbacks = callbacks,
                             verbose = 0)

plt.plot(np.arange(len(history.history['val_auroc'])),history.history['val_auroc'],label='validation')
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['auroc'],label='train')
plt.xlabel('epochs')
plt.ylabel('ROC AUC')
plt.legend()
plt.show()

print(history.history['val_auroc'][-1])

y_pred = model.predict(test_pos_matrix,
                       batch_size = batch_size)

fpr, tpr, _ = metrics.roc_curve(
    y_test,
    y_pred,
    pos_label=1,
)
auc = metrics.auc(fpr, tpr)
print(auc)

## Entity input

In [ ]:
x_input = Input(shape=train_ent_matrix.shape[1])
x_output = Embedding(input_dim = num_ent,
                    output_dim = 64)(x_input)
x_avg = GlobalAveragePooling1D()(x_output)
x_max = GlobalMaxPool1D()(x_output)
x_output = Concatenate()([x_avg,x_max])
#x_output = Dense(256,activation='relu')(x_output)
#x_output = Dense(128,activation='relu')(x_output)
x_output = Dense(64,activation='relu')(x_output)
x_output = Dense(32,activation='relu')(x_output)
x_output = Dense(16,activation='relu')(x_output)
x_output = Dense(1,activation='sigmoid')(x_output)
model = Model(inputs=x_input, outputs=x_output)
opt = Adam()
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', auroc])
model.summary()

patience = 50
batch_size = 256
epochs = 1000

callbacks = [EarlyStopping(monitor='val_auroc', patience=patience)]
history = model.fit(train_ent_matrix,y_train,
                             validation_data = (val_ent_matrix,y_val),
                             batch_size = batch_size,
                             epochs = epochs,
                             callbacks = callbacks,
                             verbose = 0)

plt.plot(np.arange(len(history.history['val_auroc'])),history.history['val_auroc'],label='validation')
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['auroc'],label='train')
plt.xlabel('epochs')
plt.ylabel('ROC AUC')
plt.legend()
plt.show()

print(history.history['val_auroc'][-1])

print(history.history['val_auroc'][-1])

y_pred = model.predict(test_ent_matrix,
                       batch_size = batch_size)

fpr, tpr, _ = metrics.roc_curve(
    y_test,
    y_pred,
    pos_label=1,
)
auc = metrics.auc(fpr, tpr)
print(auc)

## RF Input

In [ ]:
drop_out_rates = [0.1,0.2,0.3,0.4,0.5]
for drop_outs in drop_out_rates:
    x_input = Input(shape=num_features)
    x_num = Dense(256,activation='relu')(x_input)
    x_num = Dropout(drop_outs)(x_num)
    x_num = Dense(128,activation='relu')(x_num)
    x_num = Dropout(drop_outs)(x_num)
    x_num = Dense(64,activation='relu')(x_num)
    x_num = Dropout(drop_outs)(x_num)
    x_num = Dense(32,activation='relu')(x_num)
    x_output = Dense(1,activation='sigmoid')(x_num)
    model = Model(inputs=x_input, outputs=x_output)
    opt = Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', auroc])
    model.summary()

    patience = 50
    batch_size = 256
    epochs = 1000

    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_val_scaled = scaler.transform(x_val)
    x_test_scaled = scaler.transform(x_test)


    callbacks = [EarlyStopping(monitor='val_auroc', patience=patience)]
    history = model.fit(x_train_scaled,y_train,
                                 validation_data = (x_val_scaled,y_val),
                                 batch_size = batch_size,
                                 epochs = epochs,
                                 callbacks = callbacks,
                                 verbose = 0)

    plt.plot(np.arange(len(history.history['val_auroc'])),history.history['val_auroc'],label='validation')
    plt.plot(np.arange(len(history.history['val_auroc'])),history.history['auroc'],label='train')
    plt.xlabel('epochs')
    plt.ylabel('ROC AUC')
    plt.legend()
    plt.show()

    print(history.history['val_auroc'][-1])

    y_pred = model.predict(x_test_scaled,
                           batch_size = batch_size)

    fpr, tpr, _ = metrics.roc_curve(
        y_test,
        y_pred,
        pos_label=1,
    )
    auc = metrics.auc(fpr, tpr)
    print(auc)

In [ ]:
pd.DataFrame(x_train.flatten()).describe()

In [ ]:
sns.distplot(x_train.flatten())

In [ ]:
print(history.history['val_auroc'][-1])

In [ ]:
history.history.keys()

In [ ]:
x_input = Input(shape=train_ent_matrix.shape[1])
x_output = Embedding(input_dim = max_pos_len,
                    output_dim = 64)(x_input)
x_output = Dense(32,activation='relu')(x_output)
x_output = GlobalAveragePooling1D()(x_output)
x_output = Dense(1,activation='sigmoid')(x_output)
model = Model(inputs=x_input, outputs=x_output)
opt = Adam()
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', auroc])
model.summary()

In [ ]:
patience = 10
batch_size = 32
epochs = 100

callbacks = [EarlyStopping(monitor='val_loss', patience=patience)]
history = model.fit(train_ent_matrix,y_train,
                             validation_data = (val_ent_matrix,y_val),
                             batch_size = batch_size,
                             epochs = epochs,
                             callbacks = callbacks)

In [ ]:
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['val_auroc'],label='validation')
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['auroc'],label='train')
plt.xlabel('epochs')
plt.ylabel('ROC AUC')
plt.legend()
plt.show()

In [ ]:
x_input = Input(shape=x_train.shape[1])
x_output = Dense(256,activation='relu')(x_input)
x_output = Dense(128,activation='relu')(x_output)
x_output = Dense(64,activation='relu')(x_output)
x_output = Dense(1,activation='sigmoid')(x_output)
model = Model(inputs=x_input, outputs=x_output)
opt = Adam()
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', auroc])

In [ ]:
model.summary()

In [ ]:
patience = 10
batch_size = 32
epochs = 100

callbacks = [EarlyStopping(monitor='val_loss', patience=patience)]
history = model.fit(x_train,y_train,
                             validation_data = (x_val,y_val),
                             batch_size = batch_size,
                             epochs = epochs,
                             callbacks = callbacks)

In [ ]:
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['val_auroc'],label='validation')
plt.plot(np.arange(len(history.history['val_auroc'])),history.history['auroc'],label='train')
plt.xlabel('epochs')
plt.ylabel('ROC AUC')
plt.legend()
plt.show()

In [ ]:
x_train.shape

In [ ]:
np.unique(y_train,return_counts = True)